In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
import catboost as cb
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
# load data
df = pd.read_csv('/Users/shuva/Downloads/df/df/CA_housing.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [3]:
# Encode categorical values
le = LabelEncoder()
df['ocean_proximity'] = le.fit_transform(df['ocean_proximity'])

# Dealing with Nans
cols = df.columns
df = pd.DataFrame(KNNImputer(n_neighbors=5, ).fit_transform(df))
df.columns = cols

In [4]:
# train and test sample
X = df.drop("median_house_value", axis=1)
y = np.log(df["median_house_value"])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Optuna Optimization

In [5]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 1, 15),
        'random_state': 0
    }

    model = xgb.XGBRegressor(**params, )
    return cross_val_score(model, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error').mean()

In [6]:
study = optuna.create_study(direction='maximize')
%time study.optimize(objective, n_trials=30, show_progress_bar=True, n_jobs=10)

[I 2023-11-10 22:17:02,354] A new study created in memory with name: no-name-a8614284-391c-48f1-81af-b3c59ce8f6a5


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2023-11-10 22:18:01,664] Trial 5 finished with value: -0.22788116311757456 and parameters: {'n_estimators': 279, 'max_depth': 4}. Best is trial 5 with value: -0.22788116311757456.
[I 2023-11-10 22:18:27,727] Trial 1 finished with value: -0.2370917391016246 and parameters: {'n_estimators': 846, 'max_depth': 2}. Best is trial 5 with value: -0.22788116311757456.
[I 2023-11-10 22:18:32,792] Trial 4 finished with value: -0.22781062430149762 and parameters: {'n_estimators': 149, 'max_depth': 9}. Best is trial 4 with value: -0.22781062430149762.
[I 2023-11-10 22:18:39,043] Trial 11 finished with value: -0.24330072787442156 and parameters: {'n_estimators': 60, 'max_depth': 4}. Best is trial 4 with value: -0.22781062430149762.
[I 2023-11-10 22:19:04,033] Trial 7 finished with value: -0.2289366538588034 and parameters: {'n_estimators': 840, 'max_depth': 3}. Best is trial 4 with value: -0.22781062430149762.
[I 2023-11-10 22:19:38,272] Trial 9 finished with value: -0.22707485019575374 and param

In [7]:
print('Best hyperparameters:', study.best_params)
print('Best RMSE:', study.best_value)

Best hyperparameters: {'n_estimators': 887, 'max_depth': 4}
Best RMSE: -0.22641692101992178


In [8]:
mod = xgb.XGBRegressor(**study.best_params)
mod = mod.fit(X_train, y_train)

In [9]:
prediction = mod.predict(X_val)
print(mean_absolute_percentage_error(np.exp(y_val), np.exp(prediction)))

0.16154225232169692


In [10]:
#fit linear regression model
X_train = sm.add_constant(X_train)
model = sm.OLS(endog=y_train, exog=X_train).fit()

#view model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:     median_house_value   R-squared:                       0.648
Model:                            OLS   Adj. R-squared:                  0.647
Method:                 Least Squares   F-statistic:                     3369.
Date:                Fri, 10 Nov 2023   Prob (F-statistic):               0.00
Time:                        22:26:22   Log-Likelihood:                -5507.4
No. Observations:               16512   AIC:                         1.103e+04
Df Residuals:                   16502   BIC:                         1.111e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                -12.4490      0

In [11]:
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X_train.drop(columns='const').columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X_train.drop(columns='const').values, i)
                          for i in range(len(X_train.drop(columns='const').columns))]
  
print(vif_data)

              feature         VIF
0           longitude  623.011872
1            latitude  560.130880
2  housing_median_age    7.358434
3         total_rooms   30.998594
4      total_bedrooms   95.511500
5          population   16.534817
6          households   93.696216
7       median_income    8.310703
8     ocean_proximity    1.769769


Regularization, since we do have a strong multicollinearity

In [13]:
regul = sm.OLS(y_train,X_train).fit_regularized(alpha=0)
regul.params

const                 11.050254
longitude             -0.000015
latitude              -0.006457
housing_median_age     0.009289
total_rooms           -0.000076
total_bedrooms         0.000423
population            -0.000149
households             0.000589
median_income          0.224470
ocean_proximity        0.014727
dtype: float64

In [14]:
X_val = sm.add_constant(X_val)
lm_pred = regul.predict(X_val)

mean_absolute_percentage_error(np.exp(y_val), np.exp(lm_pred))

0.33159925392724093

XGBoost model shows better results than Linear regression (especially, after optimization the first)